In [ ]:
import pylab as pl
import sympy as sp
from mtm026 import *

P0 = 1          #[N]
L1 = 5          #[m]
L2 = 4.5        #[m]
a = 0.05        #[m]
t = 0.002       #[m]
E = 200e9       #[Pa]
v = 0.3

I_1 = 1.4771e-7
I_2 = 1.4771e-7

r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12 = sp.symbols('r1:17')    # Reaktionskrafter
a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12 = sp.symbols('a1:17')    # Försjutningar

K = sp.zeros(12, 12)
f = sp.zeros(12, 1)

ke1 = Ke_balk(E*I_1, L2)
ke5 = Ke_balk(E*I_2, L1)
ke2 = ke1
ke4 = ke1
ke3 = ke1
ke6 = ke5

f[13] = -P0

assem(K, ke1, dofs=[1,2,3,4])
assem(K, ke2, dofs=[3,4,5,6])
assem(K, ke3, dofs=[7,8,9,10])
assem(K, ke4, dofs=[9,10,11,12])
assem(K, ke5, dofs=[3,4,9,10])
assem(K, ke6, dofs=[5,6,11,12])


displayvar("K", K)
displayvar("f", f)

r = sp.Matrix([r1, 0, 0, 0, r5, 0, r7, 0, 0, 0, r11, 0])
a = sp.Matrix([0, a2, a3, a4, 0, a6, 0, a8, a9, a10, 0, a12])

eq_sys = sp.Eq(K*a, f + r)
display(eq_sys)

unkown = [r1, a2, a3, a4, r5, a6, r7, a8, a9, a10, r11, a12]
sol = sp.solve(eq_sys, unkown)

displayvar("Solution", sol)

<IPython.core.display.Math object>

<IPython.core.display.Math object>